In [1]:
import os 
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import pandas as pd # type: ignore

In [2]:
from utils import get_dataset, get_model_by_tag, get_prompt_template
from LLMAnnotator import LLMAnnotator

In [3]:
os.environ['OPENAI_API_KEY_CLARIN']=os.getenv("OPENAI_API_KEY_CLARIN")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

token = os.getenv("OPENAI_API_KEY_CLARIN")

In [4]:
import pandas as pd
import re

def extract_bracketed_word(df, read_col, write_col):
    """
    Wyodrębnia pierwszy wyraz objęty nawiasami [] z kolumny `read_col`
    i zapisuje go do kolumny `write_col`.
    
    :param df: DataFrame wejściowy
    :param read_col: Nazwa kolumny z której odczytywany jest tekst
    :param write_col: Nazwa kolumny do której zapisywany będzie wynik
    :return: DataFrame z nową kolumną
    """
    def extract_last_bracketed(text):
        matches = re.findall(r'\[([^\[\]]+?)\]', str(text))
        return matches[-1] if matches else None
    
    df[write_col] = df[read_col].apply(extract_last_bracketed)
    return df

In [13]:
import pandas as pd

def validate_text_column(path, df_reference):
    # Wczytaj DataFrame z pliku CSV
    df = pd.read_csv(path)
    
    # Pobierz unikalne wartości z kolumny 'text' z df_reference
    reference_texts = set(df_reference['text'].unique())
    
    # Dodaj kolumnę 'is_valid'
    df['was_in_selected_samples'] = df['text'].isin(reference_texts)
    df = extract_bracketed_word(df, 'output', 'extracted_label')
    # df.to_csv(path, index=False)
    
    return df

In [6]:
for i in [1]:
    print('')
    print('')
    print('MUMER', i)
    print('')
    print('')
    model_tag = 'llama3'  
    dataset_tag = 'go_emotions'
    prompt = 'cot_random_samples_cohere'
    temp = 0
    number_of_experimet = i
    selected_samples =  f'{model_tag}_random42_96'
    
    
    model = get_model_by_tag(model_tag, token, temp)  
    dataset_for_annotation = get_dataset(dataset_tag=dataset_tag)[700:]
    prompt_txt = get_prompt_template(dataset_tag, prompt)
    examples_for_prompt = pd.read_csv(f'./selected_samples/{dataset_tag}/{selected_samples}.csv')

    output_path = f'./resultsTEMP/{prompt}_temp{temp}/{model_tag}/{dataset_tag}_{model_tag}_{prompt}_{selected_samples}_temp{temp}_exp{number_of_experimet}.csv'
    os.makedirs(f'./resultsTEMP/{prompt}_temp{temp}/{model_tag}', exist_ok=True)
    
    
    text_col = "text"
    label_col = "label"
    output_col = "output"
    
    
    annotator = LLMAnnotator(
        model=model,
        dataset=dataset_for_annotation,
        examples_for_prompt=examples_for_prompt,
        prompt_template=prompt_txt,
        column_text=text_col,
        column_label=label_col,
        column_output=output_col
    )
    
    df = annotator.get_results(is_save=True, output_path=output_path)
    annotator.save_results(output_path=output_path)
    df_new = validate_text_column(output_path, examples_for_prompt)





MUMER 1




/home/julita/Desktop/anote-skrpyty 19.04/LLMAnnotator.py:77: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  self.chain = LLMChain(llm=self.model, prompt=self.prompt)


Nr: 0 Predicted label: To determine the most appropriate category for the given text, let's follow the steps outlined above.

**Step 1: Summarize the main topic of the text**

The main topic of the text appears to be the speaker's consideration of an opportunity or offer. They seem to be weighing the pros and cons of accepting or declining the offer.

**Step 2: Identify any key terms or phrases that hint at a specific category**

Key phrases include "Great deal" (indicating positivity) and "It will be my loss if I decline" (suggesting potential regret).

**Step 3: Compare the topic to the definitions of each category**

Based on the analysis, categories that might fit include:

* **Optimism**: The phrase "Great deal" suggests a positive outlook.
* **Desire**: The speaker seems interested in the opportunity.
* **Approval**: The tone implies acceptance or endorsement of the offer.

However, upon closer examination, none of these categories fully capture the essence of the text.

**Step 4

Failed to multipart ingest runs: Connection error caused failure to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. The content length of 367810043 bytes exceeds the maximum size limit of 20971520 bytes. ConnectionError(MaxRetryError("HTTPSConnectionPool(host='api.smith.langchain.com', port=443): Max retries exceeded with url: /runs/multipart (Caused by ProtocolError('Connection aborted.', TimeoutError('The write operation timed out')))"))
Content-Length: 367810043
API Key: lsv2_********************************************7etrace=404a2629-2769-405a-97eb-415eb8d45c75,id=404a2629-2769-405a-97eb-415eb8d45c75


In [7]:
df=pd.read_csv(output_path)
df

,text,output,logprobs,top_logprobs,original_label,was_in_selected_samples,extracted_label
0,Great deal! It will be my loss if I decline.,To determine the most appropriate category for...,"{'content': [{'token': 'To', 'logprob': -0.216...","[{'token': 'To', 'logprob': -0.216002270579338...",admiration,False,desire
1,"Really? Edit: Ah yea, I saw the original one. ...",To determine the most appropriate category for...,"{'content': [{'token': 'To', 'logprob': -0.160...","[{'token': 'To', 'logprob': -0.160430252552032...",surprise,False,neutral
2,I laughed way harder then i shouldve,To determine the most appropriate category for...,"{'content': [{'token': 'To', 'logprob': -0.144...","[{'token': 'To', 'logprob': -0.144418373703956...",amusement,False,amusement
3,I need like a drawn out visual or something,To analyze the given text and choose the most ...,"{'content': [{'token': 'To', 'logprob': -0.143...","[{'token': 'To', 'logprob': -0.143614590167999...",desire,True,desire
4,"When I see a Lynx, even if far away, oh trust ...",To determine the most appropriate category for...,"{'content': [{'token': 'To', 'logprob': -0.191...","[{'token': 'To', 'logprob': -0.191369056701660...",approval,False,love
...,...,...,...,...,...,...,...
1215,"Yeah, i dont know bout you guys, but my parent...",To determine the most appropriate category for...,"{'content': [{'token': 'To', 'logprob': -0.224...","[{'token': 'To', 'logprob': -0.224090680480003...",approval,False,neutral
1216,This is hilarious! Sounds similar to my buddy'...,To determine the most appropriate category for...,"{'content': [{'token': 'To', 'logprob': -0.152...","[{'token': 'To', 'logprob': -0.152135685086250...",amusement,False,amusement
1217,But that jumper looks hella cosy,To determine the most appropriate category for...,"{'content': [{'token': 'To', 'logprob': -0.175...","[{'token': 'To', 'logprob': -0.175881192088127...",joy,False,admiration
1218,Sorry. This thing knows whats up!,To determine the most appropriate category for...,"{'content': [{'token': 'To', 'logprob': -0.209...","[{'token': 'To', 'logprob': -0.209071293473243...",remorse,False,admiration


In [8]:
dataset_for_annotation[700:]

,text,label
1400,"I know, sorry! I didn't want to translate some...",embarrassment
1401,What does ‘open border’ mean to you? Honest qu...,confusion
1402,"Oh don’t worry, I have an oil for that",caring
1403,Im so interested in learning more about the Ga...,excitement
1404,I was very tunnel visioned at first haha didn’...,amusement
...,...,...
1915,"Yeah, i dont know bout you guys, but my parent...",approval
1916,This is hilarious! Sounds similar to my buddy'...,amusement
1917,But that jumper looks hella cosy,joy
1918,Sorry. This thing knows whats up!,remorse


In [20]:
dfff = pd.read_csv('results/cot_random_samples_cohere_temp0/llama3/go_emotions_llama3_cot_random_samples_cohere_llama3_random42_96_temp0_exp1.csv')
dfff

,Unnamed: 0,text,output,logprobs,top_logprobs,original_label,was_in_selected_samples,extracted_label
0,0,Ok.... he can *call* it the State of the Union...,To determine the most appropriate category for...,"{'content': [{'token': 'To', 'bytes': [84, 111...","[{'token': 'To', 'bytes': [84, 111], 'logprob'...",disapproval,False,disapproval
1,1,Wheres the telescope?,To determine the most appropriate category for...,"{'content': [{'token': 'To', 'bytes': [84, 111...","[{'token': 'To', 'bytes': [84, 111], 'logprob'...",curiosity,False,Curiosity
2,2,> Why create more jobs when no one is availabl...,Let's analyze the text step by step.\n\n**Step...,"{'content': [{'token': 'Let', 'bytes': [76, 10...","[{'token': 'Let', 'bytes': [76, 101, 116], 'lo...",confusion,False,amusement
3,3,[NAME] deserves to play for a proper club ffs.,To analyze the given text and determine the mo...,"{'content': [{'token': 'To', 'bytes': [84, 111...","[{'token': 'To', 'bytes': [84, 111], 'logprob'...",admiration,True,admiration
4,4,"I'm sorry, do people NOT listen to [NAME] 9th ...",Let's analyze the text step by step.\n\n**Step...,"{'content': [{'token': 'Let', 'bytes': [76, 10...","[{'token': 'Let', 'bytes': [76, 101, 116], 'lo...",gratitude,False,curiosity
...,...,...,...,...,...,...,...,...
1915,1915,"Yeah, i dont know bout you guys, but my parent...",To determine the most appropriate category for...,"{'content': [{'token': 'To', 'logprob': -0.224...","[{'token': 'To', 'logprob': -0.224090680480003...",approval,False,neutral
1916,1916,This is hilarious! Sounds similar to my buddy'...,To determine the most appropriate category for...,"{'content': [{'token': 'To', 'logprob': -0.152...","[{'token': 'To', 'logprob': -0.152135685086250...",amusement,False,amusement
1917,1917,But that jumper looks hella cosy,To determine the most appropriate category for...,"{'content': [{'token': 'To', 'logprob': -0.175...","[{'token': 'To', 'logprob': -0.175881192088127...",joy,False,admiration
1918,1918,Sorry. This thing knows whats up!,To determine the most appropriate category for...,"{'content': [{'token': 'To', 'logprob': -0.209...","[{'token': 'To', 'logprob': -0.209071293473243...",remorse,False,admiration


In [16]:
dfff = validate_text_column('results/cot_random_samples_cohere_temp0/llama3/go_emotions_llama3_cot_random_samples_cohere_llama3_random42_96_temp0_exp1.csv', examples_for_prompt)

In [17]:
df_combined = pd.concat([dfff, df], ignore_index=True)

In [18]:
df_combined

,text,output,logprobs,top_logprobs,original_label,was_in_selected_samples,extracted_label
0,Ok.... he can *call* it the State of the Union...,To determine the most appropriate category for...,"{'content': [{'token': 'To', 'bytes': [84, 111...","[{'token': 'To', 'bytes': [84, 111], 'logprob'...",disapproval,False,disapproval
1,Wheres the telescope?,To determine the most appropriate category for...,"{'content': [{'token': 'To', 'bytes': [84, 111...","[{'token': 'To', 'bytes': [84, 111], 'logprob'...",curiosity,False,Curiosity
2,> Why create more jobs when no one is availabl...,Let's analyze the text step by step.\n\n**Step...,"{'content': [{'token': 'Let', 'bytes': [76, 10...","[{'token': 'Let', 'bytes': [76, 101, 116], 'lo...",confusion,False,amusement
3,[NAME] deserves to play for a proper club ffs.,To analyze the given text and determine the mo...,"{'content': [{'token': 'To', 'bytes': [84, 111...","[{'token': 'To', 'bytes': [84, 111], 'logprob'...",admiration,True,admiration
4,"I'm sorry, do people NOT listen to [NAME] 9th ...",Let's analyze the text step by step.\n\n**Step...,"{'content': [{'token': 'Let', 'bytes': [76, 10...","[{'token': 'Let', 'bytes': [76, 101, 116], 'lo...",gratitude,False,curiosity
...,...,...,...,...,...,...,...
1915,"Yeah, i dont know bout you guys, but my parent...",To determine the most appropriate category for...,"{'content': [{'token': 'To', 'logprob': -0.224...","[{'token': 'To', 'logprob': -0.224090680480003...",approval,False,neutral
1916,This is hilarious! Sounds similar to my buddy'...,To determine the most appropriate category for...,"{'content': [{'token': 'To', 'logprob': -0.152...","[{'token': 'To', 'logprob': -0.152135685086250...",amusement,False,amusement
1917,But that jumper looks hella cosy,To determine the most appropriate category for...,"{'content': [{'token': 'To', 'logprob': -0.175...","[{'token': 'To', 'logprob': -0.175881192088127...",joy,False,admiration
1918,Sorry. This thing knows whats up!,To determine the most appropriate category for...,"{'content': [{'token': 'To', 'logprob': -0.209...","[{'token': 'To', 'logprob': -0.209071293473243...",remorse,False,admiration


In [19]:
df_combined.to_csv('results/cot_random_samples_cohere_temp0/llama3/go_emotions_llama3_cot_random_samples_cohere_llama3_random42_96_temp0_exp1.csv')